In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mitochondrial_Disorders"
cohort = "GSE65399"

# Input paths
in_trait_dir = "../../input/GEO/Mitochondrial_Disorders"
in_cohort_dir = "../../input/GEO/Mitochondrial_Disorders/GSE65399"

# Output paths
out_data_file = "../../output/preprocess/Mitochondrial_Disorders/GSE65399.csv"
out_gene_data_file = "../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE65399.csv"
out_clinical_data_file = "../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE65399.csv"
json_path = "../../output/preprocess/Mitochondrial_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE65399_family.soft.gz', 'GSE65399_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mitochondrial_Disorders/GSE65399/GSE65399_family.soft.gz
Matrix file: ../../input/GEO/Mitochondrial_Disorders/GSE65399/GSE65399_series_matrix.txt.gz


Background Information:
!Series_title	"Epigenetic therapy for Friedreich ataxia."
!Series_summary	"We set out to investigate whether a histone deacetylase inhibitor (HDACi) would be effective in an in vitro model for the neurodegenerative disease Friedreich ataxia (FRDA) and to evaluate safety and surrogate markers of efficacy in a phase I clinical trial in patients. In the neuronal cell model, HDACi 109/RG2833 increases FXN mRNA levels and frataxin protein, with concomitant changes in the epigenetic state of the gene. Chromatin signatures indicate that histone H3 lysine 9 is a key residue for gene silencing through methylation and reactivation through acetylation, mediated by the HDACi. Drug treatment in FRDA patients demonstrated increased FXN mRNA and H3 lysine 9 acetylation in peripheral blood mononuclear cells. No safety issues were encountered."
!Series_overall_design	"We used a human FRDA neuronal cell model, derived from patient induced pluripotent stem cells, to determine the 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series overall design and background information, it's clear this dataset contains gene expression data
# from the Illumina HT12v4 Gene Expression BeadArray
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:

# 2.1 Data Availability
# For trait: This is a study on Friedreich ataxia (FRDA), but there's no explicit information 
# about which samples have FRDA vs controls in the sample characteristics
trait_row = None  # No evident trait classification in sample characteristics

# For age: No age information is available in sample characteristics
age_row = None

# For gender: No gender information is available in sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since we don't have trait, age, or gender data, we'll define placeholder conversion functions
def convert_trait(value):
    """Convert trait values to binary: 1 for FRDA, 0 for control."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since we don't have trait data
    return None

def convert_age(value):
    """Convert age values to continuous numerical values."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since we don't have age data
    return None

def convert_gender(value):
    """Convert gender values to binary: 0 for female, 1 for male."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since we don't have gender data
    return None

# 3. Save Metadata
# We have gene expression data but no trait information for classification
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 47323
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers start with "ILMN_" which indicates they are Illumina BeadArray probe IDs
# These are not human gene symbols but microarray probe identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns from the gene annotation
# From the preview, we can see that 'ID' contains the probe identifiers (ILMN_*) 
# and 'Symbol' contains the gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

print(f"Using '{prob_col}' column for probe identifiers and '{gene_col}' column for gene symbols")

# 2. Get the mapping dataframe with the selected columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression dataframe shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to the specified output file
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using 'ID' column for probe identifiers and 'Symbol' column for gene symbols
Gene mapping dataframe shape: (44837, 2)
First few rows of gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB


Converted gene expression dataframe shape: (21464, 75)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE65399.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Based on the sample characteristics, there's no clear trait information related to Mitochondrial_Disorders
# The data only contains tissue types and time points, not disease state or case/control information
print("No trait information found that relates to Mitochondrial_Disorders in this dataset.")

# Since no trait information is available, we'll skip the clinical data processing
# and only save the gene expression data
# Create a minimal DataFrame with proper structure for validation
empty_df = pd.DataFrame({'placeholder': [0]})

# 5. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from Friedreich ataxia study but lacks clear trait information for " + \
       "classification of samples regarding Mitochondrial_Disorders. The sample characteristics only contain " + \
       "tissue types and time points."

# Final validation with appropriate parameters
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=True,  # Indicating dataset is biased (no variation in trait)
    df=empty_df,
    note=note
)

print("Dataset validation complete. Gene data was saved, but no linked data was created due to missing trait information.")

Shape of gene data after normalization: (20259, 75)


Saved normalized gene data to ../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE65399.csv
Sample characteristics dictionary:
{0: ['differentiation or tissue type: neural progenitors', 'differentiation or tissue type: brain fetal', 'differentiation or tissue type: undifferentiated', 'differentiation or tissue type: heart fetal', 'differentiation or tissue type: kidney fetal', 'differentiation or tissue type: liver fetal', 'differentiation or tissue type: lung fetal', 'differentiation or tissue type: pancreas fetal', 'differentiation or tissue type: small intestine fetal', 'differentiation or tissue type: stomach fetal', 'differentiation or tissue type: thymus fetal', 'differentiation or tissue type: adrenal fetal', 'differentiation or tissue type: spleen fetal'], 1: ['time point: d24', 'time point: 20wk', nan, 'time point: 18wk', 'time point: 17wk', 'time point: 10wk', 'time point: 15wk', 'time point: 14wk', 'time point: 19wk', 'time point: 8wk', 'time point: 21wk']}
No trait

Dataset validation complete. Gene data was saved, but no linked data was created due to missing trait information.
